In [ ]:
from src import object_render
from pathlib import Path
from src import model_save_load
import numpy as np
import open3d as o3d
data_dir = Path("./data/hammer_1")
depth_dir = data_dir/"depths"

In [ ]:
file_name = depth_dir/"depth_0.png"
depth_mask_file_name = depth_dir/"depth_mask_0.npy"
depth = o3d.io.read_image(file_name)

In [ ]:
depth_mask = np.load(depth_mask_file_name)
mask = depth_mask.astype("bool")
depth_array = np.asarray(depth)
masked_depth_array = depth_array * mask

In [ ]:
masked_depth_array.shape

In [ ]:
o3d.io.write_image(data_dir/"masked_depth.png",o3d.geometry.Image(masked_depth_array))

In [ ]:
1080*1920